In [1]:
import datetime
import time
from breeze_connect import BreezeConnect

In [3]:
API_KEY = ""
API_SECRET = ""
SESSION_TOKEN = ""

# --- CONFIG ---
EXCHANGE_CODE = "NFO"
STOCK_CODE = "NIFTY"
EXPIRY_DATE = "22-May-2025"
STRIKE_PRICE = "24850"
RIGHT = "call" # "put" or "call"
# --- Strategy Parameters ---
PRODUCT_TYPE = "options"
lot_size = 75
CAPITAL = 10000
PROFIT_PERCENTAGE = 2.0
STOP_LOSS_PERCENTAGE = 0.1
TRADING_START_TIME = datetime.time(9, 20)
TRADING_END_TIME = datetime.time(15, 15)

# --- State Variables ---
candle_buffer = []
current_minute = None
previous_candle_close = None
position_open = False
entry_price = None
target_price = None
stop_loss_price = None
order_quantity = 0
entry_order_id = None
last_trade_minute = None
last_minute_checked = None  
total_trades = 0
wins = 0
losses = 0
cumulative_pnl = []
timestamps = []  

# --- Candle Builder ---
def build_1min_candle(tick_data):
    global candle_buffer, current_minute

    ts = datetime.datetime.now()
    ts_minute = ts.replace(second=0, microsecond=0)
    price = float(tick_data['close'])

    if current_minute != ts_minute:
        if candle_buffer:
            open_price = candle_buffer[0]
            high_price = max(candle_buffer)
            low_price = min(candle_buffer)
            close_price = candle_buffer[-1]
            candle_data = {
                'timestamp': current_minute,
                'open': open_price,
                'high': high_price,
                'low': low_price,
                'close': close_price
            }
            on_1min_bar(candle_data)
        candle_buffer = [price]
        current_minute = ts_minute
    else:
        candle_buffer.append(price)

# --- Order Placement ---
def place_order(action, price, quantity):
    try:
        if quantity > 0:
            response = breeze.place_order(
                stock_code=STOCK_CODE,
                exchange_code=EXCHANGE_CODE,
                product=PRODUCT_TYPE,
                action=action,
                order_type="limit",
                stoploss="",
                quantity=quantity,
                price=price,
                validity="day",
                disclosed_quantity="0",
                expiry_date=EXPIRY_DATE,
                right=RIGHT,
                strike_price=STRIKE_PRICE
            )
            print(f"Order Placement Response: {response}")
            if response.get('Status') == 200 and 'Success' in response:
                print(f"✅ Order placed: ID {response['Success']['order_id']}")
                return response['Success']['order_id']
            else:
                print("❌ Order failed.")
        else:
            print("⚠️ Quantity is zero.")
        return None
    except Exception as e:
        print(f"❗ Order error: {e}")
        return None

def check_order_status(order_id, max_retries=10, delay=0.2):
    for attempt in range(max_retries):
        try:
            response = breeze.get_order_detail(exchange_code=EXCHANGE_CODE,
                                               order_id=order_id)
            print(f"Order Status Response for {order_id}: {response}")

            if response and response['Success'][0]['status'] == 'Executed':
                print("✅ Order executed.")
                return True

            print(f"⏳ Attempt {attempt+1}/{max_retries}: Order not yet executed. Retrying in {delay} seconds...")
            time.sleep(delay)

        except Exception as e:
            print(f"❗ Exception while checking order status: {e}")
            return False

    print("⚠️ Maximum retries reached. Order not executed.")
    breeze.cancel_order(exchange_code=EXCHANGE_CODE,
                        order_id=order_id)
    return False

# --- Tick Handler ---
def on_tick(tick_data):
    global position_open, entry_price, target_price, stop_loss_price, order_quantity
    global total_trades, wins, losses, cumulative_pnl, timestamps, previous_candle_close, last_minute_checked

    now = datetime.datetime.now().time()
    if now < TRADING_START_TIME or now > TRADING_END_TIME:
        return

    if 'close' not in tick_data:
        return

    ts = datetime.datetime.now()
    ts_minute = ts.replace(second=0, microsecond=0)
    price = float(tick_data['close'])

    if not position_open and previous_candle_close is not None:
        if last_minute_checked != ts_minute:
            open_price = price
            last_minute_checked = ts_minute
            if open_price > previous_candle_close:
                entry_price = open_price
                lots = int(CAPITAL / (entry_price * lot_size))
                order_quantity = lots * lot_size
                entry_order_id = place_order("BUY", entry_price, order_quantity)
                if entry_order_id and check_order_status(entry_order_id):
                    position_open = True
                    print(f"📈 BUY opened @ {entry_price}, TP: {target_price}, SL: {stop_loss_price}")
                stop_loss_price = entry_price * (1 - STOP_LOSS_PERCENTAGE / 100)
                target_price = entry_price * (1 + PROFIT_PERCENTAGE / 100)
               
                
    build_1min_candle(tick_data)

    if position_open:
        ltp = price
        if ltp >= target_price:
            square_off("target", ts)
        elif ltp <= stop_loss_price:
            square_off("stoploss", ts) #first limi9t nad next market

def square_off(reason, ts):
    global position_open, entry_price, order_quantity, total_trades, wins, losses, cumulative_pnl, timestamps
    try:
        response = breeze.square_off(
            exchange_code=EXCHANGE_CODE,
            product=PRODUCT_TYPE,
            stock_code=STOCK_CODE,
            expiry_date=EXPIRY_DATE,
            right=RIGHT,
            strike_price=STRIKE_PRICE,
            action="sell",
            order_type="market",
            validity="day",
            stoploss="",
            quantity=order_quantity,
            price="0",
            trade_password="",
            disclosed_quantity="0"
        )
        print(f"📤 Square-off ({reason}) response: {response}")
        if response.get('Status') == 200:
            pnl = (target_price - entry_price) * order_quantity if reason == "target" else (stop_loss_price - entry_price) * order_quantity
            if reason == "target":
                wins += 1
                print(f"✅ TP hit! Profit: {pnl:.2f}")
            else:
                losses += 1
                print(f"❌ SL hit! Loss: {pnl:.2f}")
            total_trades += 1
            cumulative_pnl.append(pnl)
            timestamps.append(ts)
            position_open = False
    except Exception as e:
        print(f"❗ Error during square-off: {e}")

def on_1min_bar(candle_data):
    global previous_candle_close
    previous_candle_close = candle_data['close']


breeze = BreezeConnect(api_key=API_KEY)
breeze.generate_session(api_secret=API_SECRET, session_token=SESSION_TOKEN)
print("✅ Session connected.")

breeze.ws_connect()
print("📡 WebSocket connected.")

breeze.on_ticks = on_tick
breeze.subscribe_feeds(
    exchange_code=EXCHANGE_CODE,
    stock_code=STOCK_CODE,
    expiry_date=EXPIRY_DATE,
    strike_price=STRIKE_PRICE,
    right=RIGHT,
    product_type=PRODUCT_TYPE,
    get_market_depth=False,
    get_exchange_quotes=True,
    interval="1second"
)
print("🔔 Subscribed to 1-second option feed.")

try:
    input("🔐 Press Enter to stop...\n")
finally:
    breeze.ws_disconnect()
    print("🚪 Disconnected.")
 

✅ Session connected.
📡 WebSocket connected.
🔔 Subscribed to 1-second option feed.
Order Placement Response: {'Success': {'order_id': '202505211400022786', 'message': 'Successfully Placed the order', 'user_remark': ''}, 'Status': 200, 'Error': None}
✅ Order placed: ID 202505211400022786
Order Status Response for 202505211400022786: {'Success': [{'order_id': '202505211400022786', 'exchange_order_id': '1000000022909579', 'exchange_code': 'NFO', 'stock_code': 'NIFTY', 'product_type': 'Options', 'action': 'Buy', 'order_type': 'Limit', 'stoploss': '0', 'quantity': '75', 'price': '117.9', 'validity': 'Day', 'disclosed_quantity': '0', 'expiry_date': '22-May-2025', 'right': 'Call', 'strike_price': 24850.0, 'average_price': '0', 'cancelled_quantity': '0', 'pending_quantity': '75', 'status': 'Ordered', 'user_remark': '', 'order_datetime': '21-May-2025 09:46:03', 'parent_order_id': '', 'modification_number': None, 'exchange_acknowledgement_date': None, 'SLTP_price': None, 'exchange_acknowledge_num

🔐 Press Enter to stop...
 


🚪 Disconnected.
